In [1]:
%matplotlib notebook
import tensorflow as tf
import pandas as pd
import numpy as np
import os #to view environment variables of the Operating System
import glob #find all the pathnames matching a specified pattern according to the rules used by the Unix shell

# Dataset for SOM - country level

I use indicators that are commonly used in the macroprudential literature to capture the building up of vulnerabilities.

## Inflation

**What?** Measure the rise in prices, the rate at which the general level of prices for goods and services is rising.


We implement one factor
- **Inflation** - *Inflat*

[Source](https://data.oecd.org/price/inflation-cpi.htm)




## Quarterly GDP change

**What?** Gross Domestic Product (GDP) is the standard measure of the value of final goods and services produced by a country during a period minus the value of imports. "Gross" signifies that no deduction has been made for the depreciation of machinery, buildings and other capital products used in production. "Domestic" means that it relates to the output produced on the economic territory of the country. The "products" refer to final goods and services, that is, those that are purchased, imputed or otherwise, as: the final consumption of households, non-profit institutions serving households and government; fixed capital formation; and exports (minus imports). 

We implement one factor

- **Quarterly seasonally adjusted percentage GDP change** - *QGDP*

[Source](https://data.oecd.org/gdp/quarterly-gdp.htm)

**House Price Index**

**What?** The house price index captures price changes of all residential properties purchased by households (flats, detached houses, terraced houses, etc.), both new and existing, independently of their final use and their previous owners. Only market prices are considered, self-build dwellings are therefore excluded. The land component is included. 

We implement one factor

- **House Price Index quarterly rate of change** - *HousePriceIndex*

[Source](http://ec.europa.eu/eurostat/tgm/table.do?tab=table&init=1&language=en&pcode=tipsho40&plugin=1)


## Goverment deficit to GDP

**What?** 

[Source](http://ec.europa.eu/eurostat/web/government-finance-statistics/data/database)

We implement one factor

- **Quarterly Goverment Debt** - *GenGovDeficittoGDP*




## Current account balance to GDP

**What?**  The current account balance of payments is a record of a country's international transactions with the rest of the world. The current account includes all the transactions (other than those in financial items) that involve economic values and occur between resident and non-resident entities. (Kauko (2012) - Current account to GDP as an early warning signal for banking crises)

We implement one factor

**The current account balance to GDP** - *CAbalancetoGDP*

[Source](https://data.oecd.org/trade/current-account-balance.htm)




## Credit to private sector to GDP

**What?** Domestic credit to private sector refers to financial resources provided to the private sector by financial corporations(examples of other than bank financial corporations are finance and leasing companies, money lenders, insurance corporations, pension funds, and foreign exchange companies), such as through loans, purchases of nonequity securities, and trade credits and other accounts receivable, that establish a claim for repayment.

Credit to GDP gap (defined as the difference between the credit-to-GDP ratio and its long-term trend) measures the risk associated with the credit given to householdz and busisesses in a country. A high credit to GDP gap means trouble for the banking system.

**Why?** Credit to GDP gap is defined in Basel: Under the Basel Committee on Banking Supervision (aka Basel III), the countercyclical capital buffer system of the banking system should be raised when a nation's credit-to-GDP ratio (where credit is defined as credit given to the household and private non-financial corporate sector or HH and PNFC credit) exceeds its long-term trend by two percentage points
There are the below reasons for maintaining the Counter-cyclical Capital Buffer (CCCB)
- Firstly, it requires banks to build up a buffer of capital in good times which may be used to maintain flow of credit to the real sector in difficult times.
- Secondly, it achieves the broader macro-prudential goal of restricting the banking sector from indiscriminate lending in the periods of excess credit growth that have often been associated with the building up of system-wide risk.

In actual practice healthy economies maintain the credit to GDP gap between 2 and 10 (China has an unusually high one)

We implement two factors based on this measure
- **Credit to Private non-financial sector (households and non-􏰃financial corporations) from All sectors at Market value to GDP** - *CredPrivateAllGDP*
- **Credit to Private non-financial sector (households and non-􏰃financial corporations) from Banks, total at Market value to GDP** - *CredPrivateBanksGDP*
 
[Source](https://www.bis.org/statistics/totcredit.htm)
 
- **real credit to private sector(households and non-􏰃financial corporations) to GDP: deviation from trend** - *CreditPrivateGapGDP*

[Source](https://www.bis.org/statistics/c_gaps.htm?m=6%7C380%7C670)

Both are "The break-adjusted series", they are the result of the BIS's own calculations, and were obtained by adjusting levels through standard statistical techniques described in the special feature on the long credit series of the March 2013 issue of the BIS Quarterly Review at https://www.bis.org/


('The private sector debt is the stock of liabilities held by the sectors Non-Financial corporations (S.11) and Households and Non-Profit institutions serving households (S.14_S.15). 
The instruments that are taken into account to compile private sector debt are Debt securities (F.3) and Loans (F.4). 
Data are presented in consolidated terms, i.e. do not taking into account transactions within the same sector, and expressed in % of GDP and million of national currency. 
The MIP scoreboard indicator is the stock of private sector debt in percentage of GDP. The indicator is calculated as: [PSDt/GDPt]*100. The indicative threshold of private sector debt is 133%.')








## Short-term interest rates (3 months)

**What?** Short-term interest rates are the rates at which short-term borrowings are effected between financial institutions or the rate at which short-term government paper is issued or traded in the market. Short-term interest rates are generally averages of daily rates, measured as a percentage. Short-term interest rates are based on three-month money market rates where available. 

[Source](https://data.oecd.org/interest/short-term-interest-rates.htm#indicator-chart)

We implement one factor
**3 months short-term interest rate** - *ShortIntR*





## Long-term interest rates(10 years)

**What?** Long-term interest rates refer to government bonds maturing in ten years. Rates are mainly determined by the price charged by the lender, the risk from the borrower and the fall in the capital value. Long-term interest rates are generally averages of daily rates, measured as a percentage. These interest rates are implied by the prices at which the government bonds are traded on financial markets, not the interest rates at which the loans were issued. In all cases, they refer to bonds whose capital repayment is guaranteed by governments. Long-term interest rates are one of the determinants of business investment. Low long-term interest rates encourage investment in new equipment and high interest rates discourage it. Investment is, in turn, a major source of economic growth.

[Source](https://data.oecd.org/interest/long-term-interest-rates.htm#indicator-chart)

We implement one factor
**10 years long-term interest rate** - *LongIntR*




## Indicator of syst. event
Additionally, for supervised model - create classification based on the ECB [Index of Financial Stress (CLIFS) Composite Indicator](https://sdw.ecb.europa.eu/browse.do?node=9693347). As an alternativ, I could have used ECB definition of start-end dates of crises within european countries, although using CLIFS which is non-categorical variable by default makes for better color coding later in the SOM.
- split CLIFS for each country to 4 quatiles
- assign 4th quantile as crises category
- use this classification to train logit model, which will be used as a benchmark model for accuracy, i.e. within hyperparametrization to find the SOM grid shape, SOM learning rate and SOM distance metrics 
- use this classification to fit supervised SOM (as point above, accuracy of prediction of logistic regression need to be reached to consider trained SOM further)


In [ ]:
country = pd.read_csv('/Users/martinapresnajderova/Desktop/Diplomka/SystEvents/country_factors.csv')

# Dataset for SOM - financial institutions level

**MARKET:** US Market

**INSTITUTIONS:** National banks, state member banks, and insured nonmember banks, with a special focus on 16 bank holding companies that account for more than 90 percent of all assets and deposits in the U.S. banking industry: Bank of New York Mellon, Bank of America, BB&T Corporation, Capital One, Citigroup, Fifth Third Bancorp, Goldman Sachs, JP Morgan Chase, KeyCorp, Morgan Stanley, PNC, State Street, Sun Trust, Regions Financial, U.S. Bancorp, and Wells Fargo

**FACTORS:** 
1. Numerical financial data (ratios)
 - Bank accountings data
 - Banking sector data
 - Country macrofinancial data


2. Textual news data
 - Reuters


3. Textual tweet data
 - Twitter


# Numerical financial data

Channels of Contagion 
-  Correlation: cross-sectional data on common exposures to risk factors/asset classes for tracking large-scale imbalances
-  Balance sheet contagion: network of interbank exposures, cross-holdings and liabilities + capital
 - Bank Balance sheet
-  Spirals of illiquidity: network of short-term liabilities (payables) and receivables + ‘liquidity reserves’
-  Fire sales/ feedback effects: data on portfolio holdings of financial institutions across asset classes + capital
...

Risk Monitoring Tools
- CAMEL Ratings - review past performance, used to assess the soundness of institutions and identify those requiring special supervisory attention
- Other regulatory ratios
 - UBPR\FPR reports
 - Canary ratio - review past performance, key metrics for early warning identification
 - Peer comparisons
- Internal projections of future performance
- ...



## Risk monitoring tool selected - CAMEL
Each of the 6 component ratings are based on a qualitative and quantitative analysis of specified factors and interrelationship with other components. I will leverage the risk factors identified by CAMELS system to create first part of each risk vector describing financial institution.

### Creating FINANCIAL SOM FACTORS: CAMEL = FFIEC(Call Reports + UBPR)
FFIEC provides publicly available **Reports of Condition and Income (Call Reports)** and **Uniform Bank Performance Reports (UBPRs)** for most FDIC-insured institutions which cover many factors to re-create CAMELS ratios.



#### DATA SOURCE - Federal Financial Institutions Examination Council (FFIEC) Central Data Repository (CDR) Public Data Distribution (PDD) website and its data
FFIEC collects and provides through CDC publicly available **Reports of Condition and Income (Call Reports)** and **Uniform Bank Performance Reports (UBPRs)** for most FDIC-insured institutions.

* Every national bank, state member bank, and insured nonmember bank is required by its primary federal regulator to file a Call Report as of the close of business on the last day of each calendar quarter (the report date). 
* The specific reporting requirements depend upon the size of the bank and whether it has any foreign offices. 


##### Uniform Bank Performance Report
**What is the Uniform Bank Performance Report?**  The UBPR is a multi-page financial analysis of a commercial bank or savings bank that files the Consolidated Reports of Condition and Income (Call Report). 

**What is the source of UBPR data?**  Virtually all of the dollar values, ratios, peer group averages and percentile rankings within the UBPR are computed from financial data reported by commercial banks and savings banks on the quarterly Call Report. A minor amount of structural or non-financial data comes from the banking agencies, including the number of branches and MSA codes used in UBPR ratios and peer group analysis.

**What kinds of data appear in the UBPR?**  There are five principal types of financial data contained in the UBPR.
* 1.Dollar data for individual banks appears on most pages and is **derived from the Call Report and the Summary of Deposits**. All dollar data is displayed in thousands, consistent with the standard used in the Call Report. Income and expense data that appears in the UBPR is generally year-to-date. Balance sheet data displayed in the UBPR is usually a spot or end-of-period value.

* 2.Ratio data appears on many pages. Some ratios are quite complex and involve several levels of computation. Ratios are displayed in percent format to two decimals of precision.

* 3.Peer group average data is computed for many ratios. An average of a given ratio for all banks within the peer group is presented as a benchmark to measure individual bank performance.

* 4.Percentile rankings are computed for most ratios contained in the UBPR. It is a value from 0 to 99 and reflects the percentile or percentage position of a given bank relative to other peer banks for a specific ratio.

* 5.Some structural or demographic information is displayed, including identifying information such as RSSDID number, Certificate Number, street address and holding company information.

**Which banks is the UBPR computed for?**  The UBPR is computed for all commercial banks and savings banks that file a Call Report. Due to data incompatibility, the UBPR was previously not computed for savings banks supervised by the Office of Thrift Supervision (OTS). However, with the implementation of the Dodd Frank Act and sunset of the OTS on July 21, 2011, all thrifts are required to submit Call Reports instead of Thrift Financial Reports effective March 2012, and UBPRs are available for these institutions beginning at that time

**Link to Call Data** How is the availability of UBPR data impacted by processing of the underlying Call Report?  Most data found in the UBPR is derived directly from the Call Report filed by banks. As a consequence, the availability of UBPR data is linked directly to the availability of Call Report data. Call Report data is filed by banks as of quarter end. After the Call Report is processed and validated, UBPR data will be computed within 24 hours. For many banks the UBPR will be available within 48 hours of a successful Call Report submission. 

**How Many Reports that Use UBPR Data are Available?**
There are eight reports: 
* Performance Report 
* Peer Group Average Report 
* Peer Group Average Distribution Report 
* State Average Report 
* State Average Distribution Report 
* List of Banks in Peer Group 
* Bank XBRL Document  
* Custom Peer Group Bank Report


Additional information

* The UBPR is organized by subject, e.g., earnings, balance sheet, asset quality, liquidity, and capital. 
* Each report contains data for five separate periods of time (quarters) that tie to the quarterly Call Report filed by banks. 
* The UBPR compares the performance of a given bank both against itself over time and against the performance of a group of peer banks. Peer group average and percentile ranking data provide benchmarks to measure bank performance. 
* A UBPR may be viewed online, printed, or exported to a delimited file for further use in Excel. Additionally, once implemented within the Central Data Repository, the UBPR may be exported to an eXtensible Business Reporting Language (XBRL) file for use in XBRL enabled tools.
* The principal target audience for the UBPR is federal and state banking supervisors and the banks they supervise.

In [19]:
# #Load FFIC Datasets
# ##import Stata files 
# #Change directory/folder
# path = '/Users/martinapresnajderova/Desktop/Diplomka/variables/data/20180331_ATTRIBUTES_ACTIVE'
# os.chdir(path)

# #get list of dta files
# dta_files = glob.glob('*.xml')

In [3]:
#Load FFIC UBPR Datasets
#years = ['2017', '2016', ....., '2001']
#d = {name: pd.DataFrame() for y in years}

path = '/Users/martinapresnajderova/Desktop/Diplomka/variables/data/FFIEC/UBPR/'
def dataY(year, file):
    ### Function to load datafiles for each year ###
    folders = 'FFIEC CDR Bulk All UBPR Ratios ' + str(year)
    path = '/Users/martinapresnajderova/Desktop/Diplomka/variables/data/FFIEC/UBPR/' + folders + '/'
    txt_file = path + 'FFIEC CDR UBPR Ratios ' + str(file) + " " + str(year) + '.txt'
    data = pd.read_csv(txt_file, delimiter="\t",low_memory=False, parse_dates=True, index_col='Reporting Period') 
    return(data)

#define list of relevant variables for each file
rel_var = {}   

## CAMEL

The CAMEL rating system is used to:
* Assess the soundness of financial institutions on a uniform basis
* Identify those institutions requiring special supervisory attention
* Monitor aggregate trends in overall soundness of financial institutions

### **C**apital Adequacy

From the risk management perspective, bank capital provides the institution with a "cushion" to absorb losses, it allows institutions to continue operating during adverse financial periods, thus helping to maintain confidence in the banking system and minimize liquidity concerns. Moreover capital helps restrict excessive asset growth - the minimum capital ratio requirements limit unjustified asset expansion by requiring that asset growth is funded by a commensurate amount of additional capital. 

There are three components of regulatory capital: 
* **Common equity tier 1 capital (CET1)** - Tier 1 is the most loss-absorbing form of capital, it consist of shareholders' equity and retained earnings and is easily available and used to absorb losses without any effect on the business operations.  This componnet is intended to measure bank's financial health and as a result of financial crisis, Tier 1 Capital is one of the main focus of The Basel Committee on Banking Supervision. In the past banks would have a lot of non-equity (such as hybrid debt) in their capital structures and this has proven to be insufficient and lead to developing a need for certain level of high quality capital, where higher quality capital means more loss-absorbing capacity. This greater focus on common equity, the highest quality component of a bank’s capital means that banks will be stronger, allowing them to better withstand periods of stress.
 * Under Basel III, the minimum tier 1 capital ratio is 10.5%, which is calculated by dividing the bank's tier 1 capital by its total risk-based assets.
 * It includes qualifying common stock and related surplus net of treasury stock; retained earnings; certain accumulated other comprehensive income (AOCI) elements if the institution does not make an AOCI opt-out election, plus or minus regulatory deductions or adjustments as appropriate; and qualifying common equity tier 1 minority interests. - Ref: https://www.fdic.gov/regulations/safety/manual/section2-1.pdf
 
* **Additional tier 1 capital (AT1)** - Although the main component of Tier 1 capital must be CET1 capital, Basel III allows instruments to be included in a second element of Tier 1 capital called Additional Tier 1 (AT1), if they meet certain requirements. As CET1 capital is considered the highest quality and the most effective in absorbing losses, AT1 capital is a supplementary form of Tier 1 capital, it consists of instruments that are continuous, in that there is no fixed maturity (such as preferred shares), but that *can* - and in the case of trigger event *will* - be converted into equity. Trigger event being when CET1 capital falls below a threshold.
 * It includes qualifying noncumulative perpetual preferred stock, bank-issued Small Business Lending Fund and Troubled Asset Relief Program instruments that previously qualified for tier 1 capital, and qualifying tier 1 minority interests, less certain investments in other unconsolidated financial institutions’ instruments that would otherwise qualify as additional tier 1 capital. - Ref: https://www.fdic.gov/regulations/safety/manual/section2-1.pdf


* **Tier 2 capital**

 * Tier 2 capital includes the allowance for loan and lease losses up to 1.25 percent of risk-weighted assets, qualifying preferred stock, subordinated debt, and qualifying tier 2 minority interests, less any deductions in the tier 2 instruments of an unconsolidated financial institution. - Ref: https://www.fdic.gov/regulations/safety/manual/section2-1.pdf


Key Measures
- Total risk‐based capital /risk‐weighted assets 
- Tier 1 capital/ risk‐weighted assets
- Leverage Ratio
- Capital to Assets Ratio - Shows overall capital sufficiency
- Debt to Asset Ratio - Indicates provisioning requirements on loan portfolio for current period
- Asset Growth Rates
- Capital Growth
- Cash dividends/net income


In [4]:
#########
#UBPR Page 11A – Capital Analysis: "Capital Analysis-b" (because for some reason shifted ""->"a", a->"b", c->b, d->c)
data1 = dataY("2017", "Capital Analysis-b")
#ID RSSD=Federal Reserve RSSD ID
#UBPRP742=COMMON EQUITY TIER 1 CAPITAL
#UBPRP859=the numerator of the holding company's common equity tier 1 risk-based capital ratio
#UBPRP865=ADDITIONAL TIER 1 CAPITAL
#UBPR8274=Tier 1 Capital Allowable Under the Risk-Based Capital Guidelines
#UBPR5310=Allowable Loan and Lease Loss Allowance
#UBPR8275=Allowable Tier 2 capital from Call Report Schedule RC-R
#UBPR3792=Total Risk-Based Capital
#
data1 = data1.loc[:,['ID RSSD','UBPRP742','UBPRP859','UBPRP865','UBPR8274','UBPR5310','UBPR8275','UBPR3792']]
data1['mergeID'] = data1.index.map(str) + '-' + data1['ID RSSD'].map(str)

#########
#UBPR Page 11A – Capital Analysis: "Capital Analysis-b" (because for some reason shifted ""->"a", a->"b", c->b, d->c)
data2 = dataY("2017", "Capital Analysis-a")
#'UBPRD486'= Tier 1 Leverage Capital Ratio-the relationship between a banking organization's core capital and its total assets.
#'UBPRE633'= Dividends to Net Operating Income - Cash dividends declared on common & preferred stock from Call Report Schedule RI-A divided by net operating income (income or loss before extraordinary items and other adjustments)
#WHY are we interested in Dividend policy? Dividend policy may have played a role in weakening the capital base of banks, moreover retained earnings are the primary source of the high-quality capital required to comply 
#'UBPRD487' = Tier One Risk Based Capital to Risk-Weighted Assets
#'UBPRD488'= Total Risk-Based Capital to Risk-Weighted Assets (Total Capital Ratio)
#This ratio is advocated to ensure that banks can bear a reasonable amount of losses occurring during the operations and to ascertain bank’s loss bearing capacity. 
data2 = data2.loc[:,['ID RSSD','UBPRD486','UBPRE633','UBPRD487','UBPRD488']]
data2['mergeID'] = data2.index.map(str) + '-' + data2['ID RSSD'].map(str)

rel_var['Capital Analysis-a'] = ['ID RSSD','UBPRD486','UBPRE633','UBPRD487','UBPRD488']

#########
#UBPR Page 1 – Summary Ratios:
data3 = dataY("2017", "Summary Ratios")
#'UBPR7316'= Total assets - The annual % change from the prior year comparable quarter to the current quarter in total assets
#'UBPRE027'= Net loans and leases -- 12-month growth rate AQ
#'UBPR7408'= Tier One Capital 12-month growth rate. The percentage is determined by subtracting the account balance as 
#of the corresponding reporting period in the previous year from the current period account balance and dividing the 
#result by the previous year balance.

#'UBPRE541'= Total loans and leases 90+ days past due to gross loans and leases AQ.-The sum of loans and lease 
#financing receivables past due at least 90 days, and still in accrual status, divided by gross loans and lease-financing receivables outstanding.
#'UBPR7414'= Noncurrent loans and leases to gross loans and leases.-The sum of loans and lease financing receivables 
#past due at least 90 days, plus those in nonaccrual status, divided by gross loans and lease-financing receivables outstanding.
#'UBPRE023' = Loan and lease allowance to net loss (times) - Ending balance of the allowance for possible loan and lease losses divided by total loans and lease-financing receivables.
#'UBPRE542' = Nonaccrual loans to total loans and leases - Total loans and leases on nonaccrual status divided by total loans and leases.
#'UBPRE006' = Provision for loan and lease losses to average assets - Provision for loan and lease receivables losses divided by (as percentage of) average assets
#'UBPRE024'= Net Loans and Leases to Total Assets
#
data3 = data3.loc[:,['ID RSSD','UBPR7316','UBPRE027','UBPR7408','UBPRE541','UBPR7414','UBPRE023','UBPRE542','UBPRE006','UBPRE024']]
data3['mergeID'] = data3.index.map(str) + '-' + data3['ID RSSD'].map(str)

rel_var['Summary Ratios'] = ['ID RSSD','UBPR7316','UBPRE027','UBPR7408','UBPRE541','UBPR7414','UBPRE023','UBPRE542','UBPRE006','UBPRE024']

data = data2.merge(data3, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)



### **A**sset Quality

The quality of assets is significant factor to assess the financial strength of a bank. Risks to the solvency often arise from an impairment of assets, and for banks in particular the profitability is expressed through the quality of loan book. The deterioration in the financial health of the borrowers can misbalance the asset composition and therefore **the ratio of nonperforming loans (NPLs) to total loans** is often used as a proxy for measuring the asset quality. Assesment of non-performing loans - loans and leases 90+ days past due, nonaccrual and noncurrent loans, also give us insights into the composition and diversification of loan portfolio, where mainly the ratio between secured and total advances plays an important role, the greater the security against loans the lesser is the risk.


Asset Quality - also referred to as Credit Risk key measures:
- ALLL(=Allowances for loan and Lease losses)/Net Loan Losses
- Total loans and leases 90+ days past due to gross loans and leases
- Nonaccrual Loans/Total
- Noncurrent loans and leases to gross loans and leases 
- Net loans and leases - 12-month growth rate 
- Loan Loss Provision Ratio: Provision for loan and lease losses to average assets AQ
 * Purpose: Indicates provisioning requirements on loan portfolio for current period
 * Definition: Loan Loss Provision - Allocation in current period to the loan loss reserve.
- Total assets - annual change
- Net Loans and Leases to Total Assets


In [5]:
#########
#UBPR Page 1 – Summary Ratios:
data4 = dataY("2017", "Summary Ratios")

##Credit risk KPI purpose - How prepared is the management?
#'UBPRE021'= Loan and Lease Allowance to Net Loss

##Credit risk KPI purpose - What amount of losses do we expect? - Projected
#Already in CA - data3 - 'UBPRE541'= Total Loans and Leases 90+ Days Past Due(and still in accrual status) to Gross Loans and Leases
#Already in CA - data3 - 'UBPRE542'= Total loans and leases on nonaccrual status divided by total loans and leases
#Already in CA - data3 - 'UBPR7414'= Noncurrent Loans(The sum of loans and lease financing receivables past due at least 90 days, plus those in nonaccrual status) and Leases to Gross Loans and Leases 

#Already in CA - data3 - 'UBPRE027'= Net Loans and Leases 12-month growth rate
#Already in CA - data3 - 'UBPRE006'= Provision for Loan & Lease Losses as a percent of Average Assets
#Already in CA - data3 - 'UBPR7316'= Total assets - The annual % change from the prior year comparable quarter to the current quarter in total assets
#Already in CA - data3 - 'UBPRE024'= Net Loans and Leases to Total Assets

#########
#UBPR Page 7 – Analysis of Credit Allowance and Loan Mix:
data5 = dataY("2017", "Allowance and Loan Mix-a")

##Credit risk KPI purpose - What has been the loss experience? - History
#'UBPRE388'=Recoveries to Prior Credit Loss- Gross credit recoveries in the current year divided by gross credit losses of the preceding year
#'UBPRE019'=Net loss to average total LN&LS - Net Loss as a percent of Average Total Loans and Leases
#'UBPRE390'=Gross loss to average total LN&LS
#'UBPRE391'=Recoveries to avg tot LN&LS

##Credit risk KPI purpose - How prepared is the management?
#'UBPRE006' Loss Provision to average assets
#Already in CA - data3 - 'UBPRE023'=LN&LS Allowance to total LN&LS
#'UBPRE020'=Earnings coverage of net loss


data4 = data4.loc[:,['ID RSSD','UBPRE021']]
data4['mergeID'] = data4.index.map(str) + '-' + data4['ID RSSD'].map(str)
rel_var['Allowance and Loan Mix-a'] = ['ID RSSD','UBPRE021']


data5 = data5.loc[:,['ID RSSD','UBPRE388','UBPRE019', 'UBPRE390', 'UBPRE391','UBPRE020']]
data5['mergeID'] = data5.index.map(str) + '-' + data5['ID RSSD'].map(str)
rel_var['Summary Ratios'] = rel_var['Summary Ratios'] + ['UBPR7316','UBPRE027','UBPR7408','UBPRE541','UBPR7414','UBPRE023','UBPRE542','UBPRE006','UBPRE024']



data = data.merge(data4, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)
data = data.merge(data5, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

### **M**anagement

Management assessment determines whether an institution is able to properly react to financial stress. It is reflected by the management's capability to point out, measure, look after and control risks of the institution's daily activities. It covers the management's ability to ensure the safe operation of the institution as they comply with the necessary and applicable internal and external regulations. It is reflected by operating policies and processes and whether the institution has adequate controls.

Key measures
- Efficiency ratio
- personnel Expense as a percent of Average Assets



In [6]:
#########
#UBPR Page 3 – Noninterest Income, Expenses and Yields:
data6 = dataY("2017", "Non Int Inc, Exp, Yields")

#'UBPRE088'= Efficiancy Ratio-Total Overhead Expense expressed as a percentage of Net Interest Income (TE) plus Noninterest Income
#'UBPR7400'=Personnel Expense as a percent of Average Assets

data6 = data6.loc[:,['ID RSSD','UBPRE088','UBPR7400']]
data6['mergeID'] = data6.index.map(str) + '-' + data6['ID RSSD'].map(str)
rel_var['Non Int Inc, Exp, Yields'] = ['ID RSSD','UBPRE088','UBPR7400']


data = data.merge(data6, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

### **E**arnings

Ability to create appropriate returns to be able to expand, retain competitiveness, and add capital company's growth, stability, valuation allowances, net interest margin, net worth level and the quality of the company's existing assets.

The KPIs should focus on the level as well as the quality of earnings.

Key measures:
- Cash dividends to net income
- Retained earnings to average total equity
- Pretax operating income (TE) to average assets
- Net income to average assets (Return on Performing Assets-ROA)
 * Indicates financial productivity of credit services and investments activities
- Net interest income/average assets
- Non-interest income/average assets
- Non-interest expense /average assets
- Net interest income/average earning assets 
- Interest expense/average earning assets
- Interest income/average earning assets
- Net income/average equity

In [7]:
#########
#UBPR Page 1 – Summary Ratios:
data7 = dataY("2017", "Summary Ratios")
#'UBPR7402'=Cash Dividends to Net Income(If net income is less than or equal to zero, NA is shown at this caption)
#'UBPRE025'=Retained Earnings to Avg Total Equity: Net income, less cash dividends declared, divided by average equity 
#capital
#'UBPRE007'=Pretax Operating Income (TE) as a percent of Average Assets:Net interest income on a tax-equivalent basis 
#plus noninterest income, less noninterest expense, the provision for loan and lease-financing receivable losses and 
#the provision for allocated transfer risk, divided by average assets.
#'UBPRE013'=Net Income as a percent of Average Assets
#'UBPRE003'=Net Interest Income (TE) as a percent of Average Assets
#'UBPRE004'=Noninterest Income as a percent of Average Assets
#'UBPRE005'=Non-Interest Expense as a percent of Average Assets: Salaries and employee benefits, expenses of premises 
#and fixed assets and other noninterest expense divided by average assets.
#'UBPRE018'=Net Interest Income (TE) as a percent of Average Earning Assets:Total interest income on a tax-equivalent 
#basis, less total interest expense, divided by the average of the respective asset accounts involved in generating 
#interest income.
#'UBPRE017'=Interest Expense to Average Earning Assets
#'UBPRE016'=Interest Income (TE) as a percent of Average Earning Assets

data7 = data7.loc[:,['ID RSSD','UBPR7402','UBPRE025', 'UBPRE007', 'UBPRE013','UBPRE003', 'UBPRE004', 'UBPRE005','UBPRE018','UBPRE017','UBPRE016' ]]
data7['mergeID'] = data7.index.map(str) + '-' + data7['ID RSSD'].map(str)
rel_var['Summary Ratios'] = rel_var['Summary Ratios'] + ['UBPR7402','UBPRE025', 'UBPRE007', 'UBPRE013','UBPRE003', 'UBPRE004', 'UBPRE005','UBPRE018','UBPRE017','UBPRE016']



data = data.merge(data7, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)


#########
#UBPR Page 11 – Capital Analysis:
data8 = dataY("2017", "Capital Analysis-a")
#'UBPRE630'=Net Income as a Percent of Average Bank Equity Capital

data8 = data8.loc[:,['ID RSSD','UBPRE630']]
data8['mergeID'] = data8.index.map(str) + '-' + data8['ID RSSD'].map(str)
rel_var['Capital Analysis-a'] = rel_var['Capital Analysis-a'] + ['UBPRE630']



data = data.merge(data8, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

### **L**iquidity

Interest rate risk sensitivity, availability of assets which can easily be converted to cash, dependence on short-term volatile financial resources and ALM technical competence.

Heavy emphasis on cash flow based liquidity projections
Liquidity Risk - 'The variation in net income and market value of equity caused by a bank's difficulty in obtaining cash at a reasonable cost from either the sale of assets or new borrowings.'


Banks can acquire liquidity in two distinct ways:
By liquidation of assets 
 - Composition of investments
 - Maturity of investments
 
By borrowing 
 - Core deposits
 - Volatile deposits



Key measures
- Average interest-bearing funds to average assets
- Short-term non-core funding - 12-month growth rate
- Short-term investments - 12-month growth rate
- Cash dividends/Net income 
- Net loans and leases to deposits
- Brokered deposits/total deposits 
- Net noncore funding dependence
 * Typical measure of contingency liquidity, measures the relationship between long-term earning assets and net short-term funds.
 * Dependence on noncore funding sources(federal funds purchased, subordinated notes and debentures, large CDs,large CDs, small brokered deposits, foreign deposits and other borrowings) for larger banks has been increasing with time, with the evolution of financial services, depositors have the opportunity to invest within these higher expected return alternatives and shift away from banks. In the meantime, the composition of bank portfolio is growing in loans. To keep up, banks have therefore turned to more nontraditional noncore sources of funds.
 * Non-core funding dependence is the difference between non-core liabilities and short-term investments, divided by long-term assets. This ratio is based on the premise that non-core liabilities are better suited to fund short-term investments rather than long-term assets.
- Core deposits to assets
- Short Term Investments as a percent of Short Term Non Core Funding



In [8]:
#########
#UBPR Page 1 – Summary Ratios:
data9 = dataY("2017", "Summary Ratios")
#'UBPRE015'=Average Interest-Bearing Funds as a percent of Average Assets
#'UBPRE029'=Short Term Non Core Funding 12-month growth rate
#'UBPRE028'=Short Term Investments 12-month growth rate
#Already in Earnings -'UBPR7402'=Cash Dividends to Net Income

data9 = data9.loc[:,['ID RSSD','UBPRE015','UBPRE029','UBPRE028']]
data9['mergeID'] = data9.index.map(str) + '-' + data9['ID RSSD'].map(str)
rel_var['Summary Ratios'] = rel_var.get('Summary Ratios', []) + ['UBPRE015','UBPRE029','UBPRE028']

data = data.merge(data9, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#########
#UBPR Page 10 – Liquidity and Funding:
data10 = dataY("2017", "Liquidity and Funding")
#'UBPRE600'=Net Loans & Leases as a percent of Total Deposits
#'UBPRE595'=Total Brokered Deposits as a percent of Total Deposits
#'UBPRK447'=Net Non Core Funding Dependence $250,000: (Noncore liabilities - short term investments)/long term assets
#Non core liabilities = sum of total time deposits of more than $250,000 + other borrowed money + foreign office deposits + securities sold under agreements to repurchase + federal funds purchased + insured brokered deposits less than $100,000 + insured brokered deposits of $100,000 through $250,000.
#Short term investments = sum of: interest-bearing bank balances + federal funds sold + securities purchased under agreements to resell + debt securities with a remaining maturity of one year or less. 
#Long term assets = sum of: net loans and leases + loans and leases held for sale + held-to-maturity securities + available-for-sale securities - debt securities with a remaining maturity of one year or less + other real estate owned (non-investment).
#'UBPRE591'=Core Deposits as a percent of Total Assets
#'UBPRE597'=Short Term Investments as a percent of Short Term Non Core Funding

data10 = data10.loc[:,['ID RSSD','UBPRE600','UBPRE595','UBPRK447','UBPRE591','UBPRE597']]
data10['mergeID'] = data10.index.map(str) + '-' + data10['ID RSSD'].map(str)
rel_var['Liquidity and Funding'] = rel_var.get('Liquidity and Funding',[]) + ['ID RSSD','UBPRE600','UBPRE595','UBPRK447','UBPRE591','UBPRE597']


data = data.merge(data10, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

Now, let's gather the accounting and financial ratios for financial institution from the Federal Deposit Insurance Corporation UBPR reports for each of the defined range of years and create representatives of the CAMELS rating system used by bank regulators to take into account different dimensions of bank condition.


In [9]:
path = '/Users/martinapresnajderova/Desktop/Diplomka/variables/data/FFIEC/UBPR/'

#redefine dataY function, now with RSSD mapping dictionary 'rel_var'
def dataY2(year, file):
    ### Function to load datafiles for each year ###
    folders = 'FFIEC CDR Bulk All UBPR Ratios ' + str(year)
    path = '/Users/martinapresnajderova/Desktop/Diplomka/variables/data/FFIEC/UBPR/' + folders + '/'
    txt_file = path + 'FFIEC CDR UBPR Ratios ' + str(file) + " " + str(year) + '.txt'
    data = pd.read_csv(txt_file, delimiter="\t",low_memory=False, parse_dates=True, index_col='Reporting Period')
    #add merge_ID
    data['mergeID'] = data.index.map(str) + '-' + data['ID RSSD'].map(str)
    data = data.loc[:,['mergeID'] + rel_var[file]]
    return(data)


#2003
year=2003
data2003 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2003 = data2003.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2004
year=2004
data2004 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2004 = data2004.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2005
year=2005
data2005 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2005 = data2005.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2006
year=2006
data2006 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2006 = data2006.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2007
year=2007
data2007 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2007 = data2007.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2008
year=2008
data2008 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2008 = data2008.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2009
year=2009
data2009 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2009 = data2009.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2010
year=2010
data2010 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2010 = data2010.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2011
year=2011
data2011 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2011 = data2011.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2012
year=2012
data2012 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2012 = data2012.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2013
year=2013
data2013 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2013 = data2013.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2014
year=2014
data2014 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2014 = data2014.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2015
year=2015
data2015 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2015 = data2015.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2016
year=2016
data2016 = pd.DataFrame({'mergeID':[], 'ID RSSD':[]})
for file in rel_var:
    data_add = dataY2(str(year), str(file))
    data2016 = data2016.merge(data_add, how='outer', on=['mergeID', 'ID RSSD'], left_index=True)

#2017
data2017 = data

In [34]:
#all_years = ['data{}'.format(item) for item in range(2003,2017)]
#mydata = pd.concat([data2003, data2004, data2005, data2006, data2007, data2008, data2009, data2010, data2011, data2012, data2013, data2014, data2015, data2016])
data2004 = data2004.drop(data2004.index[[0]])
data2005 = data2005.drop(data2005.index[[0]])
data2006 = data2006.drop(data2006.index[[0]])
data2007 = data2007.drop(data2007.index[[0]])
data2008 = data2008.drop(data2008.index[[0]])
data2009 = data2009.drop(data2009.index[[0]])
data2010 = data2010.drop(data2010.index[[0]])
data2011 = data2011.drop(data2011.index[[0]])
data2012 = data2012.drop(data2012.index[[0]])
data2013 = data2013.drop(data2013.index[[0]])
data2014 = data2014.drop(data2014.index[[0]])
data2015 = data2015.drop(data2015.index[[0]])
data2016 = data2016.drop(data2016.index[[0]])

mydata = pd.concat([data2003, data2004, data2005, data2006, data2007, data2008, data2009, data2010, data2011, data2012, data2013, data2014, data2015, data2016])


In [35]:
mydata.to_csv('/Users/martinapresnajderova/Desktop/Diplomka/variables/mydata.csv')

# List of financial institutions

In [10]:
#Map RSSD IDs to the institutions: names and atributes 
#https://www.newyorkfed.org/research/banking_research/datasets.html
#includes 1,412 PERMCO-RSSD links from June 30, 1986 to December 31, 2016
mapping = pd.read_csv('/Users/martinapresnajderova/Desktop/Diplomka/variables/RSSDNameMap.csv')

#Focus on 16 banks
#16 bank holding companies that account for more than 90 percent of all assets and deposits in the U.S. banking industry: 
#Bank of New York Mellon, Bank of America, BB&T Corporation, Capital One, Citigroup, Fifth Third Bancorp, Goldman Sachs, JP Morgan Chase, KeyCorp, Morgan Stanley, PNC, State Street, Sun Trust, Regions Financial, U.S. Bancorp, and Wells Fargo

banks_rssd = {"Bank of New York Mellon": 3587146, "Bank of America":1073757, "BB_T Corporation":1074156, "Capital One":2277860, "Citigroup":1951350, "Fifth Third Bancorp":1070345, "Goldman Sachs":2380443, "JP Morgan Chase":1039502, "KeyCorp":1034806, "Morgan Stanley":2162966, "PNC, State Street":1069778, "Sun Trust":1131787, "Regions Financial_2004":1078332,"Regions Financial_2016":3242838, "U.S. Bancorp":1119794, "Wells Fargo":1120754}
banks_reuters = {"Bank of New York Mellon": ['BNY Mellon', 'Bank of New York Mellon', 'BK.N'], "Bank of America":['BAC.N', 'Bank of America', 'Bank of America\'s'], "BB_T Corporation":['BB&T Corp', 'BB&T'], "Capital One":['COF.N', 'Capital One'], "Citigroup":['Citigroup', 'CIT2.L'], "Fifth Third Bancorp":['Fifth Third', 'FITB.OQ'], "Goldman Sachs":['Goldman', 'GS.N'], "JP Morgan Chase":['JP Morgan'], "KeyCorp":['KEY.N', 'KeyCorp'], "Morgan Stanley":['MS.N', 'Morgan Stanley'], "PNC, State Street":['PNC'], "Sun Trust":['Suntrust', 'SunTrust', 'STI.N'],"Regions Financial":['Regions', 'Regions Financial', 'RF.N'], "U.S. Bancorp":['Bancorp', 'USB.N'], "Wells Fargo":['Wells Fargo', 'WFC.N']}

# path = '/Users/martinapresnajderova/Desktop/Diplomka/variables/data/FED/20180331_ATTRIBUTES_ACTIVE'
# os.chdir(path)
# #get list of xml files
# xml_files = glob.glob('*.XML')
# k = xml_files

# from xml.dom import minidom
# # parse an xml file by name
# mydoc = minidom.parse(k[0])
# # one specific item attribute
# print('Item #2 attribute:')  
# print(items[1].attributes['NM_LGL'].value)
# len(np.unique(data['ID RSSD']))
#print(data['ID RSSD'].unique)




# Textual Data
* **Sentiment analysis** we will be taking news articles(unstructured text) for a particular company, and we will attempt to grade this news to see how postive, negative or neutral it is.
* **Factors**
 - number of times financial institution has been mentioned in the news/twitter
 - number of times financial institution have been mentioned together in the same news/tweet


**Sources of Data**
* News - Reuters articles
* Tweets - Twitter


In [2]:
#scrape articles archives url from reuters, API doesn't work
textdata = pd.read_csv('/Users/martinapresnajderova/Desktop/Diplomka/variables/TextDataLink.csv')
textdata.head()

,Quarter,Link,Date,Bank of New York Mellon,Bank of America,BB_T Corporation,Capital One,Citigroup,Fifth Third Bancorp,Goldman Sachs,JP Morgan Chase,KeyCorp,Morgan Stanley,"PNC, State Street",Sun Trust,Regions Financial,U.S. Bancorp,Wells Fargo
0,2007Q1,https://www.reuters.com/resources/archive/us/2...,2007Month01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007Q1,https://www.reuters.com/resources/archive/us/2...,2007Month02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007Q1,https://www.reuters.com/resources/archive/us/2...,2007Month02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007Q1,https://www.reuters.com/resources/archive/us/2...,2007Month02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007Q1,https://www.reuters.com/resources/archive/us/2...,2007Month02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import nltk
import requests
from bs4 import BeautifulSoup

#create sentiment of article description for each set of articles for quarter + tag them by companies
#output:quarter, sentiment score for each article, tag companies
#sentiment engine: once we have the text articles (BeautifulSoup), pass them to sentiment engine - TextBlob package (higher level abstraction package that sits on top of NLTK)

textdata['news_keywords'] = '-'
textdata['bank'] = '-'
textdata['sentiment'] = int(0)

#https://github.com/gyanesh-m/Sentiment-analysis-of-financial-news-data/blob/master/sentiment.py

#Sentiment Engine
#2 possibilities: Dictionary approach(Vader) or ML approach

##Get text corpus = large body of text - corpora designed to contain a careful balance of material of genres
###For example: The Reuters Corpus 
####contains 10,788 news documents totaling 1.3 million words
####the documents have been classified into 90 topics, and grouped into two sets, called "training" and "test"
####This split is for training and testing algorithms that automatically detect the topic of a document
### Or SentiWordNet - sentiment scores for 145k WordNet synonym sets

##OR get lexicon = collection of words,phrases along with associated information such as part of speech and sense definitions. 
#Lexical resources are secondary to texts, and are usually created and enriched with the help of texts

#If dictionary - research groups are showing that in order to get a decent accuracy it is mandatory to use a dictionary of words developed for the specific financial purpose. 

#Download the lexicon file and install Vader!
#import nltk
#nltk.downloader.download()
#d vader_lexicon
#q
#Vader (Valence Aware Dictionary and sEntiment Reasoner) - https://github.com/cjhutto/vaderSentiment
#The VADER algorithm outputs sentiment scores to 4 classes of sentiments 
#neg: Negative
#neu: Neutral
#pos: Positive
#compound: Compound (i.e. aggregated score)

# from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
# sentences=["hello","why is it not working?!"]
# sid = SIA()
# for sentence in sentences:
#     ss = sid.polarity_scores(sentence) 


# print(ss)



In [ ]:
#text information for the most significant banks
#codes used within the reuters articles for the institutions
banks_reuters = {"Bank of New York Mellon": ['BNY Mellon', 'Bank of New York Mellon', 'BK.N'], "Bank of America":['BAC.N', 'Bank of America', 'Bank of America\'s'], "BB_T Corporation":['BB&T Corp', 'BB&T'], "Capital One":['COF.N', 'Capital One'], "Citigroup":['Citigroup', 'CIT2.L'], "Fifth Third Bancorp":['Fifth Third', 'FITB.OQ'], "Goldman Sachs":['Goldman', 'GS.N'], "JP Morgan Chase":['JP Morgan'], "KeyCorp":['KEY.N', 'KeyCorp'], "Morgan Stanley":['MS.N', 'Morgan Stanley'], "PNC, State Street":['PNC'], "Sun Trust":['Suntrust', 'SunTrust', 'STI.N'],"Regions Financial":['Regions', 'Regions Financial', 'RF.N'], "U.S. Bancorp":['Bancorp', 'USB.N'], "Wells Fargo":['Wells Fargo', 'WFC.N']}

       
def sentiment(inp):
    bank = tag[0]
    score = 0
    r = requests.get(inp)
    b = BeautifulSoup(r.content,'html.parser')
    #get list of article links for each archive
    links = b.select('div > div > div > div > div > div > a')
    #open each link, extract article, extract company tag, run sentiment analysis, output score
    sent_score = np.empty([1, 1], dtype=float)
    for link in links:
        try:
            r1 = requests.get(link.get('href'))
            if (r1.status_code==200): 
                b1 = BeautifulSoup(r1.text)
                tags = b1.findAll(attrs={"name": "news_keywords"})[0]['content']
                if [item for item in tag if item in tags] != []:
                    #look for bank in bank tag dictionary
                    desc = b1.findAll(attrs={"name": "description"})
                    #sentiment
                    sid = SIA()
                    ss = sid.polarity_scores(desc[0]['content'].encode('utf-8'))
                    ss = ss['compound'] #The compound score=sum of all of the lexicon ratings standardised to range between -1 and 1
                    sent_score = np.append(sent_score, ss)
        except Exception as e:
            print(e)
    score = int(np.nansum(sent_score) / float(len(sent_score)))
    return(score)



#test
banks_reuters = {"Bank of New York Mellon": ['Bank of New York Mellon','BNY Mellon', 'Bank of New York Mellon', 'BK.N'], "Bank of America":['BAC.N', 'Bank of America', 'Bank of America\'s'], "BB_T Corporation":['BB&T Corp', 'BB&T'], "Capital One":['COF.N', 'Capital One'], "Citigroup":['Citigroup', 'CIT2.L'], "Fifth Third Bancorp":['Fifth Third', 'FITB.OQ'], "Goldman Sachs":['Goldman', 'GS.N'], "JP Morgan Chase":['JP Morgan'], "KeyCorp":['KEY.N', 'KeyCorp'], "Morgan Stanley":['MS.N', 'Morgan Stanley'], "PNC, State Street":['PNC'], "Sun Trust":['Suntrust', 'SunTrust', 'STI.N'],"Regions Financial":['Regions', 'Regions Financial', 'RF.N'], "U.S. Bancorp":['Bancorp', 'USB.N'], "Wells Fargo":['Wells Fargo', 'WFC.N']}
#sentiment('https://www.reuters.com/resources/archive/us/20070101.html',tag=banks_reuters["Bank of New York Mellon"])

#apply on whole text data for each company - each bank has its columns for sent_score
#tag=banks_reuters["Bank of New York Mellon"]
#textdata['Bank of New York Mellon'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Bank of America"]
# textdata['Bank of America'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["BB_T Corporation"]
# textdata['BB_T Corporation'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Capital One"]
# textdata['Capital One'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Citigroup"]
# textdata['Citigroup'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Fifth Third Bancorp"]
# textdata['Fifth Third Bancorp'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Goldman Sachs"]
# textdata['Goldman Sachs'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["JP Morgan Chase"]
# textdata['JP Morgan Chase'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["KeyCorp"]
# textdata['KeyCorp'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Morgan Stanley"]
# textdata['Morgan Stanley'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["PNC, State Street"]
# textdata['PNC, State Street'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Sun Trust"]
# textdata['Sun Trust'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Regions Financial"]
# textdata['Regions Financial'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["U.S. Bancorp"]
# textdata['U.S. Bancorp'] = textdata['Link'].apply(sentiment)

# tag=banks_reuters["Wells Fargo"]
# textdata['Wells Fargo'] = textdata['Link'].apply(sentiment)

# banks_reuters["Bank of America"]
# banks_reuters["BB_T Corporation"]
# banks_reuters["Capital One"]
# banks_reuters["Citigroup"]
# banks_reuters["Fifth Third Bancorp"]
# banks_reuters["Goldman Sachs"]
# banks_reuters["JP Morgan Chase"]
# banks_reuters["KeyCorp"]
# banks_reuters["Morgan Stanley"]
# banks_reuters["PNC, State Street"]
# banks_reuters["Sun Trust"]
# banks_reuters["Regions Financial"]
# banks_reuters["U.S. Bancorp"]
# banks_reuters["Wells Fargo"]